# Quickstart: Vector Search for Beginners

### Initialize the client

In [1]:
from qdrant_client import QdrantClient

client = QdrantClient(url="http://localhost:6333")

/home/joviedo/github-repos/llm-zoomcamp-knowledge-base/week2/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Create a collection

In [3]:
from qdrant_client.models import Distance, VectorParams

client.create_collection(
    collection_name="test_collection",
    vectors_config=VectorParams(size=4, distance=Distance.DOT),
)

UnexpectedResponse: Unexpected Response: 409 (Conflict)
Raw response content:
b'{"status":{"error":"Wrong input: Collection `test_collection` already exists!"},"time":0.000078299}'

### Add vectors

In [4]:
from qdrant_client.models import PointStruct

operation_info = client.upsert(
    collection_name="test_collection",
    wait=True,
    points=[
        PointStruct(id=1, vector=[0.05, 0.61, 0.76, 0.74], payload={"city": "Berlin"}),
        PointStruct(id=2, vector=[0.19, 0.81, 0.75, 0.11], payload={"city": "London"}),
        PointStruct(id=3, vector=[0.36, 0.55, 0.47, 0.94], payload={"city": "Moscow"}),
        PointStruct(id=4, vector=[0.18, 0.01, 0.85, 0.80], payload={"city": "New York"}),
        PointStruct(id=5, vector=[0.24, 0.18, 0.22, 0.44], payload={"city": "Beijing"}),
        PointStruct(id=6, vector=[0.35, 0.08, 0.11, 0.44], payload={"city": "Mumbai"}),
    ],
)

print(operation_info)

operation_id=0 status=<UpdateStatus.COMPLETED: 'completed'>


### Run a query

Let’s ask a basic question - Which of our stored vectors are most similar to the query vector [0.2, 0.1, 0.9, 0.7]?

In [20]:
search_result = client.query_points(
    collection_name="test_collection",
    query=[0.2, 0.1, 0.9, 0.7],
    with_payload=False,
    limit=3
).points

print(*search_result, sep='\n')

id=4 version=0 score=1.362 payload=None vector=None shard_key=None order_value=None
id=1 version=0 score=1.273 payload=None vector=None shard_key=None order_value=None
id=3 version=0 score=1.208 payload=None vector=None shard_key=None order_value=None


### Add a filter

We can narrow down the results further by filtering by payload. Let’s find the closest results that include “London”.

In [18]:
from qdrant_client.models import Filter, FieldCondition, MatchValue

search_result = client.query_points(
    collection_name="test_collection",
    query=[0.2, 0.1, 0.9, 0.7],
    query_filter=Filter(
        must=[FieldCondition(key="city", match=MatchValue(value="London"))]
    ),
    with_payload=True,
    limit=3,
).points

print(*search_result, sep='\n')

id=2 version=0 score=0.871 payload={'city': 'London'} vector=None shard_key=None order_value=None
